Import all packages

In [ ]:
import pyarrow as pa
from pyarrow import json
import litendb as ten

Use Liten cache to store local data efficiently.

In [ ]:
tc = ten.Cache()

In [ ]:
import os
from huggingface_hub import snapshot_download
dataset='musical-instruments'
local_dir = f'./{dataset}'
if not os.path.exists(local_dir):
    print(f'Downloading dataset into {local_dir} ...')
    snapshot_download(
        repo_id="hkverma/"+dataset,
        repo_type="dataset",
        local_dir=local_dir
    )
    print('Download Complete.')
else:
    print(f"Dataset already exists in {local_dir}, skipping download.")  

In [ ]:
reviews_file=f'{local_dir}/Musical_Instruments.json'
reviews_pa = pa.json.read_json(input_file=reviews_file)
reviews_table = tc.add_table("reviews_all", reviews_pa, tc.FactTable)
reviews_table

In [ ]:
reviews_pa = reviews_pa.drop(['reviewerID','asin','helpful','unixReviewTime','reviewTime'])
reviews_table = tc.add_table("reviews", reviews_pa, tc.FactTable)
tc.slice('reviews',0,5).to_pandas()

In [ ]:
from textblob import TextBlob

In [ ]:
reviews_pa = tc.slice('reviews',0,12)
review_texts = reviews_pa.to_pandas()['reviewText']       

In [ ]:
review_texts

In [ ]:
review_sentiments = []
review_sentiments_text = []
for review_text in review_texts:
    blob = TextBlob(review_text)
    review_sentiments.append(blob.sentiment.polarity)
    if (blob.sentiment.polarity > 0.5):
        review_sentiment_text = "+++"
    elif (blob.sentiment.polarity > 0.25):
        review_sentiment_text = "++"
    elif (blob.sentiment.polarity >= 0):
        review_sentiment_text = "+"
    elif (blob.sentiment.polarity > -0.25):
        review_sentiment_text = "-"
    elif (blob.sentiment.polarity > -0.5):
        review_sentiment_text = "--"
    else:
        review_sentiment_text = "---"
    review_sentiments_text.append(review_sentiment_text)

In [ ]:
reviews_pa = reviews_pa.append_column('sentiment',pa.array(review_sentiments))
reviews_pa = reviews_pa.append_column('sentiment_text',pa.array(review_sentiments_text))

In [ ]:
reviews_pa.to_pandas()